In [1]:
from pyomo.environ import *

In [4]:
model=ConcreteModel()
model.x_1=Var(within=NonNegativeReals)
model.x_2=Var(within=NonNegativeReals)
model.obj=Objective(expr=model.x_1+2*model.x_2)
model.con1=Constraint(expr=3*model.x_1+4*model.x_2>=1)
model.con2=Constraint(expr=2*model.x_1+5*model.x_2>=2)

In [8]:
N=[1,2]
M=[1,2]
c={1:1,2:2}
a={(1,1):3,(1,2):4,(2,1):2,(2,2):5}
b={1:1,2:2}

In [ ]:
model.x=Var(N,within=NonnegativeReals)

def obj_rule(model):
    return sum(c[i]*model.x[i] for i in N)